# Text Analytics - Sentiment

## Import modules, variable assignment
The first step of the process is to import the required packages and assign variable values.

In [80]:
import requests
import json

## Variables to assign
sasserver = "https://viya-cq9hud3gec.engage.sas.com"
text = " Today it rained; we didn't go to school\r\n456, Our team won the game\r\n789, The funeral was a sad event\r\n012, The quick brown fox jumped over the lazy dog\r\n345, What a long strange trip it's been\r\n678, The telephone was rang and I handed it to Liz. She said \"This isn't who it would be If it wasn't who it is\"\r\n901, She was having a no good terrible very bad day\r\n234, If I could be the sun I'd radiate like Africa and Smile upon the world Intergalactic love laughter"
access_token = "eyJqa3UiOiJodHRwczovL2xvY2FsaG9zdC9TQVNMb2dvbi90b2tlbl9rZXlzIiwia2lkIjoibGVnYWN5LXRva2VuLWtleSIsInR5cCI6IkpXVCIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJkNDFhODllYy01NGRjLTQyZGMtOWUxMS02NmQ3NmVlMjFkYTMiLCJzZXNzaW9uX3NpZyI6IjAwZDMxMzdhLTNlOTEtNGRhNy04ODU2LTA3OTkzZDczZDBmMiIsInVzZXJfbmFtZSI6ImpvZS5mdXJiZWVAc2FzLmNvbSIsIm9yaWdpbiI6ImV4dGVybmFsX29hdXRoIiwiaXNzIjoiaHR0cDovL2xvY2FsaG9zdC9TQVNMb2dvbi9vYXV0aC90b2tlbiIsImF1dGhvcml0aWVzIjpbIkRhdGFCdWlsZGVycyIsIlNjaGVkdWxlU2VydmljZUFjY291bnRVc2VycyIsIkFwcGxpY2F0aW9uQWRtaW5pc3RyYXRvcnMiLCJCYXRjaFNlcnZpY2VBY2NvdW50VXNlcnMiLCJMYXVuY2hlclN1cGVyVXNlcnMiLCJFc3JpVXNlcnMiLCJEYXRhQWdlbnRBZG1pbmlzdHJhdG9ycyIsIlNDSU0iLCJEYXRhQWdlbnRQb3dlclVzZXJzIiwiU0FTU2NvcmVVc2VycyIsIkdsb3NzYXJ5Lkdsb3NzYXJ5QWRtaW5pc3RyYXRvcnMiLCJDYXRhbG9nLlN1YmplY3RNYXR0ZXJFeHBlcnRzIiwiQ29tcHV0ZVNlcnZpY2VBY2NvdW50VXNlcnMiLCJDQVNIb3N0QWNjb3VudFJlcXVpcmVkIl0sImNsaWVudF9pZCI6ImpodWJ1c2VyIiwiYXVkIjpbImpodWJ1c2VyIiwib3BlbmlkIl0sImV4dF9pZCI6IjAwdTJzbHk2YzZTb3c2T1J5MnA3IiwicmVtb3RlX2lwIjoiMTQ5LjE3My44LjM5IiwiemlkIjoidWFhIiwiZ3JhbnRfdHlwZSI6ImF1dGhvcml6YXRpb25fY29kZSIsInVzZXJfaWQiOiJkNDFhODllYy01NGRjLTQyZGMtOWUxMS02NmQ3NmVlMjFkYTMiLCJhenAiOiJqaHVidXNlciIsInNjb3BlIjpbIm9wZW5pZCJdLCJhdXRoX3RpbWUiOjE3NDIzMDkzMTAsImV4cCI6MTc0MzYwNTMxMCwiaWF0IjoxNzQyMzA5MzEwLCJqdGkiOiJiNmY0MWE1ODA3NDI0OTA4OTlmYWMxNjhhYjQ0NzY3MCIsImVtYWlsIjoiam9lLmZ1cmJlZUBzYXMuY29tIiwicmV2X3NpZyI6ImQ1Y2I0MDgxIiwiY2lkIjoiamh1YnVzZXIifQ.bA9EDNv7cblOfTbb6cPjZmvp7IYbM92fNQtXdkgoG-guNBOBqlFjv5NAkkI0wRFHmpVICReODkC3GwLtDczTOYkimEGi_H5rJzB4Gl02YRwNeDvqULaL10h57z4OqGLNluHb5ykjSTrIwm0ygiVqODWPA1I_Z3YezoZ0xiLrKj9AcaUHIjRwqutzxPWmmFhatULM7fdifbc0snG9k7kNnc4QU38xCFHXy5PHTl8bq-fcFzJ8ltwkjQort54QDeRQbkgdgX2GvqwLfI5b3aSHqw5BL4CtCXbP0fGQwdDUoUtR54YsxmfbYNaXvzJvBnWvbnej0o6KtTKCwFL7uvPP7A" # Get from the authentication project

## Create CAS connection to start making calls

In [81]:
import swat, os
conn = swat.CAS(os.environ['CAS_CONTROLLER'], 5570, password=os.environ['ACCESS_TOKEN'])
conn

CAS('controller.sas-cas-server-default.viya', 5570, 'joe.furbee@sas.com', protocol='cas', name='py-session-6', session='fb028c7f-e562-fc46-a37c-5d377b788395')

In [82]:
# Get session id and print it out
sessionId = conn.sessionid().session
sessionId

'fb028c7f-e562-fc46-a37c-5d377b788395'

## Load data from the form to be scored

In [83]:
#Upload data function
def csv_load(sessionId, access_token, sasserver, data):

   url = sasserver + "/cas-shared-default-http/cas/sessions/"+sessionId+"/actions/upload"
   payload = "UID,Text\r\n"+data

   headers = {
      'Accept': 'application/json',
      'Content-Type': 'binary/octet-stream',
      'JSON-Parameters': '{"casout":{"caslib":"casuser","name":"tableToScore","replace":true},"importOptions":{"fileType":"csv"}}',
      'Authorization': 'Bearer '+ access_token
   }


   try:
      response = requests.request("PUT", url, headers=headers, data = payload, verify=False)
      return str(response)
   except requests.exceptions.RequestException as e:
      return e

In [84]:
# Upload Data
unique_id = 123
parsedData = str(unique_id)+","+text
print(parsedData)

# Load CSV
load_data = csv_load(sessionId, access_token, sasserver, parsedData)
print(load_data)

123, Today it rained; we didn't go to school
456, Our team won the game
789, The funeral was a sad event
012, The quick brown fox jumped over the lazy dog
345, What a long strange trip it's been
678, The telephone was rang and I handed it to Liz. She said "This isn't who it would be If it wasn't who it is"
901, She was having a no good terrible very bad day
234, If I could be the sun I'd radiate like Africa and Smile upon the world Intergalactic love laughter


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'viya-cq9hud3gec.engage.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>


## Run Sentiment Model

In [76]:
# Run Sentiment Model function
def callSentimentModel(sessionId, access_token, sasserver):
   headers = {
      "Accept": "application/json",
      "Content-Type": "application/json",
      "Authorization": "Bearer " + access_token }

   url = sasserver + '/cas-shared-default-http/cas/sessions/'+sessionId+'/actions/sentimentAnalysis.applySent'

   payload = "{"table":{"caslib":"casuser", "name":"tableToScore"},\r\n\"docId\":\"uid\",\r\n\"text\":\"text\",\r\n\"language\":\"ENGLISH\",\r\n\"casOut\":{\"caslib\":\"casuser\", \"name\":\"sentimentAnalysis\", \"promote\":true}}"
#   payload = "{\"table\":{\"caslib\":\"casuser\", \"name\":\"tableToScore\"},\r\n\"docId\":\"uid\",\r\n\"text\":\"text\",\r\n\"language\":\"ENGLISH\",\r\n\"casOut\":{\"caslib\":\"casuser\", \"name\":\"sentimentAnalysis\", \"promote\":true}}"

   try:
      response = requests.request("POST", url, headers=headers, data=payload, verify=False).json()
      return(response)
   except requests.exceptions.RequestException as e:
      return e

In [77]:
# Run Sentiment Model
sentiment_output = callSentimentModel(sessionId, access_token, sasserver)
sentiment_output_format = json.dumps(sentiment_output, indent=2)
print(sentiment_output_format) # optional print statement to view output

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'viya-cq9hud3gec.engage.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{
  "changedResources": [
    "tables",
    "actionsets"
  ],
  "disposition": {
    "debugInfo": null,
    "formattedStatus": null,
    "reason": "OK",
    "severity": "Normal",
    "statusCode": 0
  },
  "log": "NOTE: Added action set 'sentimentAnalysis'.\n",
  "logEntries": [
    {
      "level": "info",
      "message": "NOTE: Added action set 'sentimentAnalysis'."
    }
  ],
  "metrics": {
    "cpuSystemTime": 0.340726,
    "cpuUserTime": 0.66469,
    "dataMovementBytes": 22429800,
    "elapsedTime": 0.570871,
    "involuntaryContextSwitches": 0,
    "ioPageFaults": 0,
    "ioPageReclaims": 0,
    "ioReadBlocks": 0,
    "ioReadBytes": 0,
    "ioReadCalls": 0,
    "ioReadFaults": 0,
    "ioWriteBytes": 0,
    "ioWriteBytesCancelled": 0,
    "ioWriteCalls": 0,
    "memory": 873056,
    "memoryQuota": 185499648,
    "osMemory": 85778432,
    "systemCores": 40,
    "systemMemory": 0,
    "systemNodes": 3,
    "systemTotalMemory": 337560707072,
    "voluntaryContextSwitches": 0
  },
  

## Get the data using the Fetch action

In [78]:
# Get Sentiment data function
def getData(sasserver, sessionId, access_token, tblName):
   url = sasserver + "/cas-shared-default-http/cas/sessions/" + sessionId + "/actions/table.fetch"

   payload = "{ \"table\": {\"caslib\":\"casuser\", \"name\":\"" + tblName + "\"} }"

   headers = {
      'Authorization': "Bearer " + access_token,
      'Content-Type': 'application/json'
   }

   try:
      response = requests.request("POST", url, headers=headers, data = payload, verify=False).json()
      return(response)
   except requests.exceptions.RequestException as e:
      return e    

In [79]:
# Get Sentiment data
sentiment_data = getData(sasserver, sessionId, access_token, "sentimentAnalysis")
sentiment_format = json.dumps(sentiment_data, indent=2)
print(sentiment_format)

# Write results to a json file
with open("sentiment_data.json", "w") as file:
    file.write(str(sentiment_format))

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'viya-cq9hud3gec.engage.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{
  "changedResources": [],
  "disposition": {
    "debugInfo": null,
    "formattedStatus": null,
    "reason": "OK",
    "severity": "Normal",
    "statusCode": 0
  },
  "log": "",
  "logEntries": [],
  "metrics": {
    "cpuSystemTime": 0.008453,
    "cpuUserTime": 0.003795,
    "dataMovementBytes": 0,
    "elapsedTime": 0.024814,
    "involuntaryContextSwitches": 0,
    "ioPageFaults": 0,
    "ioPageReclaims": 0,
    "ioReadBlocks": 0,
    "ioReadBytes": 0,
    "ioReadCalls": 0,
    "ioReadFaults": 0,
    "ioWriteBytes": 0,
    "ioWriteBytesCancelled": 0,
    "ioWriteCalls": 0,
    "memory": 4157472,
    "memoryQuota": 185499648,
    "osMemory": 37310464,
    "systemCores": 40,
    "systemMemory": 0,
    "systemNodes": 3,
    "systemTotalMemory": 337560707072,
    "voluntaryContextSwitches": 0
  },
  "results": {
    "Fetch": {
      "_ctb": true,
      "attributes": {
        "Action": {
          "type": "string",
          "value": "fetch"
        },
        "Actionset": {
      

## Drop table at end of use case

In [53]:
# Drop table function
def dropTable(sasserver, sessionId, access_token):
   url = sasserver + "/cas-shared-default-http/cas/sessions/" + sessionId + "/actions/table.dropTable"

   payload = "{\"caslib\": \"casuser\", \"name\": \"sentimentanalysis\"}"

   headers = {
       'Authorization': 'Bearer '+ access_token,
       'Content-Type': 'application/json'
   }

   try:
      response = requests.request("POST", url, headers=headers, data=payload, verify=False).json
      return(response)
   except requests.exceptions.RequestException as e:
      return e

In [54]:
# Drop table
drop_table = dropTable(sasserver, sessionId, access_token)

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'viya-cq9hud3gec.engage.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## End session

In [66]:
# End session function
def endSession(sasserver, sessionId, access_token):
   url = sasserver + "/cas-shared-default-http/cas/sessions/" + sessionId + "/actions"

   headers = {
      'Authorization': "Bearer " + access_token,
      'Content-Type': 'application/json'
   }

   try:
      response = requests.request("DELETE", url, headers=headers, verify=False).json
      return(response)
   except requests.exceptions.RequestException as e:
      return e

In [67]:
# End session
endSession= endSession(sasserver, sessionId, access_token)
print(endSession)

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'viya-cq9hud3gec.engage.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<bound method Response.json of <Response [200]>>
